# Validation
In this notebook we will validate model datasets against the GridClim product, or any other gridded observations.
There are four metrics available in `attribution.validation`
- The average anomaly
- The monthly average anomaly
- A seasonal correlation index
- A spatial correlation index

In [1]:
import glob
import os
from importlib import reload

import attribution.validation
import dask
import iris
import iris.plot as iplt
import numpy as np
import pandas as pd
from attribution.config import init_config
from dask.distributed import Client
from matplotlib import pyplot as plt

/home/sm_erhol/dev/attribution/attribution/bootstrap.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
client = Client()

In [3]:
CFG = init_config()

In [4]:
# Where do we store the data?
project_folder = CFG["paths"]["project_folder"]

In [5]:
# We are investigating multiple variables, have to select one.
variables = CFG["variable"]

In [6]:
# Which variable will we use?
variable = variables[0]
print(variable)

pr


In [7]:
gc_path = glob.glob(os.path.join(project_folder,  f"{variable}*GridClim*.nc"))
cordex_path = glob.glob(os.path.join(project_folder, f"{variable}*CORDEX*.nc"))

In [8]:
# GridClim
gc_cube = iris.load_cube(gc_path)
# Cordex
cordex_cube = iris.load_cube(cordex_path)

/tmp/ipykernel_32641/298512951.py:2: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  gc_cube = iris.load_cube(gc_path)
/tmp/ipykernel_32641/298512951.py:4: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  cordex_cube = iris.load_cube(cordex_path)


## Average (daily) anomaly
This calculates the average day anomaly between the model and reference dataset.

In [11]:
average_anomaly = attribution.validation.average_anomaly(cordex_cube, gc_cube)

/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/analysis/cartography.py:412: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2216: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2216: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))


Note that since the cube data is lazy, the above step doesn't actually perform the calculation.
This happens first when we need the data, like below.

In [12]:
# Print the average anomaly as a percentage.
average_anomaly.data * 100

masked_array(data=[ 2.73024677,  5.34765306,  5.0492481 ,  4.03765773,
                    5.14837942,  8.33328035,  7.64792237,  5.63265473,
                    6.64541918,  3.12672223,  5.06970701,  8.24756509,
                    5.59473476,  8.11843258,  8.49943123,  3.35360196,
                    3.92149371,  3.13900884,  5.52315596,  7.01755846,
                   10.96545841,  4.96435482,  4.65552442,  5.28880422,
                    5.202161  ,  7.4850673 ,  6.70678455,  3.62128417,
                    9.75481565, 11.23050514,  3.01530987, 10.86494727,
                    9.24063937,  9.07293029,  7.56184571,  6.12237315,
                    7.91313435,  4.59744212,  7.78034103,  7.76679841,
                    8.20862878,  5.71652912,  3.76276166,  8.04859561,
                    2.77032909,  3.63861118,  4.27800814,  2.50554404,
                    6.15792129,  3.51961243,  5.39361175,  1.99665449,
                    5.10045915,  8.5031181 ,  3.9609933 ,  5.75521634,
      

## Average monthly anomaly


In [13]:
average_monthly_anomaly = attribution.validation.average_monthly_anomaly(
    cordex_cube, gc_cube
)

/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2234: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2234: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'year'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/analysis/cartography.py:412: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2216: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2216: UserWarning: Collapsing a

In [14]:
# The average seasonal anomaly as a percentage.
average_monthly_anomaly.data * 100

masked_array(data=[14.99772823, 12.96316064, 12.70325731, 17.36801649,
                   12.84681757, 12.81253312, 13.01390125, 16.30853709,
                   15.0152931 , 14.72837613, 14.51891114, 12.4550897 ,
                   15.03658814, 16.78340087, 12.79907044, 14.51986758,
                   13.51995618, 11.10967744, 13.11226172, 15.04480882,
                   13.49406988, 14.7920711 , 14.58762605, 14.02206568,
                   12.35736111, 14.11011752, 12.38302202, 13.30070883,
                   15.20944323, 16.58847189,  9.8783545 , 13.17085547,
                   15.50770294, 13.81730712, 14.06663366, 14.37847521,
                   14.71514852, 10.25736903, 15.11926157, 13.05925881,
                   16.0468872 , 12.16598257, 10.87483485, 19.55669138,
                   11.29789386, 14.076847  , 11.93218012, 13.20865528,
                   12.64607436, 12.39472163, 12.59652425, 10.00945756,
                   12.79896335, 16.2312004 , 11.43776889, 13.68847647,
      

## Seasonal correlation index

There are essentially two indices we can calculate here, depending on how hard we want to test the model.
If we want to test the models ability to simulate the average year we don't need to set the arg `climatological` since this is `True` by default.
This will correlate the average annual cycle for the model(s) and observations in each grid point.
Doing this checks how well the model captures the average annual cycle for each grid point.

By setting `climatological=False` every annual cycle is instead correlated i.e. monthly values.
This is a much tougher metric, and more computationally demanding.

There are also two version of this metric, one which follows the definition in the report "Das Bayerische Klimaprojektionensemble", and one which more directly follow the Kling-Gupta efficiency.
This is switched with the `kge` kwarg.

In [15]:
seasonality_index = attribution.validation.seasonality_index(
    cordex_cube, gc_cube, kge=False
)
seasonality_index_kge = attribution.validation.seasonality_index(
    cordex_cube, gc_cube, kge=True
)

/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2234: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2234: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'year'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2234: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2234: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'year'.
  warnings.warn(msg.format(self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2234: UserWarni

In [16]:
seasonality_index.data

masked_array(data=[0.77910106, 0.81656819, 0.81292823, 0.69280485,
                   0.79249511, 0.77006811, 0.78851797, 0.68847425,
                   0.80621654, 0.76426925, 0.78009404, 0.71618689,
                   0.78837021, 0.76333477, 0.8429574 , 0.75426751,
                   0.82380766, 0.87006145, 0.84898379, 0.79993869,
                   0.79993505, 0.81665673, 0.75884914, 0.81935417,
                   0.87454689, 0.85843801, 0.85583839, 0.77184752,
                   0.85433616, 0.83091334, 0.87472736, 0.73453693,
                   0.80375639, 0.85854206, 0.83628429, 0.80786756,
                   0.78617336, 0.88808634, 0.80189531, 0.88111012,
                   0.82663456, 0.84266608, 0.87237473, 0.62284224,
                   0.83609616, 0.77167402, 0.84362893, 0.78755385,
                   0.85545783, 0.82627993, 0.85349398, 0.78627149,
                   0.84659543, 0.75721219, 0.8218243 , 0.77876741,
                   0.77229988, 0.81954119, 0.87177032, 0.82584

In [17]:
seasonality_index_kge.data

masked_array(data=[0.67050463, 0.72582064, 0.72600878, 0.57739786,
                   0.70023533, 0.72132161, 0.72513269, 0.62849035,
                   0.66536196, 0.67806864, 0.67414779, 0.74882291,
                   0.63037534, 0.62772341, 0.74347001, 0.61326681,
                   0.70897777, 0.74290681, 0.72826915, 0.6646154 ,
                   0.72602977, 0.66482342, 0.66133303, 0.71406214,
                   0.75464486, 0.72777494, 0.74331351, 0.70966711,
                   0.71001224, 0.67300281, 0.7670871 , 0.72488748,
                   0.69295729, 0.73791506, 0.72367003, 0.71378207,
                   0.67853586, 0.78681774, 0.69629186, 0.71339047,
                   0.65812006, 0.76622708, 0.77673764, 0.50582837,
                   0.74927849, 0.70302107, 0.75503559, 0.70990533,
                   0.73625543, 0.74421852, 0.75148655, 0.78526212,
                   0.75669692, 0.635303  , 0.77801519, 0.68577852,
                   0.64202313, 0.70852113, 0.75251461, 0.70813

## Spatial correlation
Measures the spatial correlation between model and reference dataset.

In [18]:
pattern_index = attribution.validation.pattern_index(cordex_cube, gc_cube)

/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2226: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'grid_latitude', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'grid_latitude'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2226: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'grid_longitude', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'grid_longitude'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/home/sm_erhol/.conda/envs/erhol/lib/python3.10/site-packages/iris/coords.py:2216: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.forma

In [19]:
pattern_index.data

masked_array(data=[0.5619961 , 0.50890505, 0.6583811 , 0.53872424,
                   0.5700649 , 0.51023006, 0.5275761 , 0.6564171 ,
                   0.577764  , 0.58950925, 0.61938363, 0.54765534,
                   0.54936165, 0.5649752 , 0.51307815, 0.5803684 ,
                   0.59288925, 0.6210491 , 0.64479727, 0.5941583 ,
                   0.5587453 , 0.630385  , 0.5691504 , 0.54878193,
                   0.6210678 , 0.6372913 , 0.60631675, 0.58400023,
                   0.6604443 , 0.6036584 , 0.5436205 , 0.5494969 ,
                   0.5820571 , 0.5620515 , 0.5529612 , 0.5826314 ,
                   0.5583478 , 0.59790504, 0.6341298 , 0.54150146,
                   0.59591085, 0.6113226 , 0.622307  , 0.59025   ,
                   0.5719771 , 0.65362245, 0.6247244 , 0.6198506 ,
                   0.6513086 , 0.5961437 , 0.64280003, 0.5938447 ,
                   0.5805416 , 0.60515195, 0.6175601 , 0.58116215,
                   0.6044712 , 0.62968475, 0.52534187, 0.54080

## Homogenise the scores
Mapping the index to scores between 0 and 10.
The bins used are based on values from the paper/report on the Bavarian climate projection ensemble.

In [20]:
# Scores for index 1 and 2
indx_1_scores = attribution.validation.get_scores(
    average_anomaly.data * 100,
    bins=np.arange(5, 51, 5),
    score_bins=np.arange(10, -1, -1),
)
indx_2_scores = attribution.validation.get_scores(
    average_monthly_anomaly.data * 100,
    bins=np.arange(5, 51, 5),
    score_bins=np.arange(10, -1, -1),
)

In [21]:
# Scores for index 3 and 4
indx_3_scores = attribution.validation.get_scores(
    seasonality_index.data,
    bins=np.arange(0.92, 0.19, -0.08),
    score_bins=np.arange(10, -1, -1),
)
indx_4_scores = attribution.validation.get_scores(
    pattern_index.data,
    bins=np.arange(0.92, 0.19, -0.08),
    score_bins=np.arange(10, -1, -1),
)

In [22]:
# Stack the scores
data = np.stack(
    [
        indx_1_scores,
        indx_2_scores,
        indx_3_scores,
        indx_4_scores,
    ],
    axis=1,
)

Create a `pandas.Dataframe` of the scores

In [23]:
# Create a dataframe of the scores.
scores_df = pd.DataFrame(data=data, columns=["Idx_1", "Idx_2", "Idx_3", "Idx_4"])
# Add the ensemble id
scores_df["ensemble_id"] = cordex_cube.coord("ensemble_id").points
scores_df = scores_df[["ensemble_id", "Idx_1", "Idx_2", "Idx_3", "Idx_4"]]

In [24]:
# Add a sum of the scores.
scores_df["Sum_scores"] = scores_df.sum(axis=1, numeric_only=True)

In [25]:
scores_df.min()

ensemble_id    CCCma-CanESM2--CLMcom-CCLM4-8-17--r1i1p1
Idx_1                                                 8
Idx_2                                                 7
Idx_3                                                 6
Idx_4                                                 4
Sum_scores                                           27
dtype: object

With these scores we can evaluate if there are any models that should be excluded from the rest of the study.
In this particular case, the lowest total score is 32, which is still acceptable, and we don't exclude any models based on this step.

In [26]:
# Save the scores to a csv.
scores_df.to_csv(os.path.join(project_folder, "scores.csv"))

In [28]:
client.shutdown()

## Next step

[Attribution](4_attribution.ipynb)